In [1]:

#Stay at bower ticket handler. Ticket is read, and male state changes to at bower. 
#The tickethandler can generate then generate a repair bower, foraging, or maurad travel ticket.
def SBtickethandler(SB_tic):
    ow=SB_tic['owner'] #the owner of the ticket is the male bird from the stay at bower ticket
    if male_states[ow]!=0: #because any male who is at his bower was last repairing his bower (not coming from foraging or maurading) 
        bower_states[ow]=1 #now the bower is intact, so the bower state is updated to reflect this change
    t=SB_tic['time'] 
    ta=SB_tic['tau']
    if bower_states[ow]==0: #if male returns to a destroyed bower, must repair it
        RB_tau=nexttau(-7) #-7 denotes a repair bower action
        SB_tic['tau']=0 #because bower repair begins immediately after returning to a destroyed bower
        timeline[timeholder]=SB_tic #change the tau on the timeline (not just SB_tic, a copy)
        RB_time=ta+t
        future_t=RB_tau+RB_time
        if future_t>t_max:
            RB_tau=RB_tau-(future_t-t_max) #this ensures that the repairing the bower tau+t won't exceed t_max 
        #RB_tau=Cutoff(t_max,MT_FG_time, nexttau(-7)) #-7 denotes a repair bower action
        tic=ticketgenerator(RB_tau,t,ow,-7,ow, t) #immediately, RB_tic starts
        addtotimeline(tic,timeline)
        male_states[ow]=[tic['time']+RB_tau, tic['action'], timeholder] #male state has 3 components: [time, action, timeholder]
    else: #if the bower is intact
        MT_FG_time=ta+t #the maurad travel or foraging time is equal to the stay at bower tau plus time
        if MT_FG_time<t_max: 
            decider=random.random()
            if decider<MT_vs_FG: #transition to MT
                targ=numpy.random.choice(list(range(nodes)), p=visit_preferences[ow]) #choosing the male to maurad based on visit preferences
                #MT_tau=Cutoff(t_max,MT_FG_time, travel_times[ow][targ]) #this ensures that the repairing the bower tau+t won't exceed t_max     
                tic=ticketgenerator(MT_tau,MT_FG_time,ow,-4, targ, t)
                male_states[ow]=[MT_FG_time, tic['action'], timeholder] #important!!
                addtotimeline(tic,timeline)
            else:
                #FG_tau=Cutoff(t_max,MT_FG_time, FG_tau, nexttau(-2))
                tic=ticketgenerator(FG_tau,MT_FG_time,ow,-2, ow, t) #-2 denotes foraging action
                male_states[ow]=[MT_FG_time, tic['action'], timeholder] #important!!
                addtotimeline(tic,timeline)
        else: #if the ticket would be late
            male_states[ow]=[t_max, -10, timeholder] #still need to show male is at bower, but -10 action is gibberish (since male won't leave bower before tmax)
            
            
#Foraging ticket handler. Ticket read, and male state changes to no longer at bower. Generates a stay at bower ticket.
def FGtickethandler(FG_tic):
    ow=FG_tic['owner'] 
    t=FG_tic['time']
    ta=FG_tic['tau']
    #print(male_states[ow])
    if t==0: #in this case male_states[ow]=0
        print("in t==0 (FGth)")
        #male_states[ow]=0 #male is no longer at bower
        SB_time=t+ta
        SB_tau=nexttau(-1)
        if SB_time<t_max:
            future_t=SB_tau+SB_time
            if future_t>t_max:
                SB_tau=SB_tau-(future_t-t_max)               
            SB_tic=ticketgenerator(SB_tau,SB_time,ow,-1, ow, t) #-1 denotes staying at bower
            addtotimeline(SB_tic,timeline)      
    elif male_states[ow][0]!=t:
        discards.append(FG_tic)
        timeline[timeholder]['action']=abs(FG_tic['action']) 
    else:
        male_states[ow]=0 #male is no longer at bower
        SB_time=t+ta
        SB_tau=nexttau(-1)
        if SB_time<t_max:
            future_t=SB_tau+SB_time
            if future_t>t_max:
                SB_tau=SB_tau-(future_t-t_max)               
            SB_tic=ticketgenerator(SB_tau,SB_time,ow,-1, ow, t) #-1 denotes staying at bower
            addtotimeline(SB_tic,timeline)
        
#Maurad travel ticket handler. Male state changes to no longer at bower, and generates a maurad action ticket.
def MTtickethandler(MT_tic):
    ow=MT_tic['owner']
    targ=MT_tic['target']
    t=MT_tic['time']
    ta=MT_tic['tau']
    if male_states[ow][0]!=t:
        discards.append(MT_tic)
        timeline[timeholder]['action']=abs(MT_tic['action']) #A CHANGE
    else:
        male_states[ow]=0 #male is no longer at his bower
        MA_time=t+ta
        if MA_time<t_max:
            MA_tau=nexttau(-5)
            future_t=MA_tau+MA_time
            if future_t>t_max:
                MA_tau=MA_tau-(future_t-t_max)   
            MA_tic=ticketgenerator(MA_tau,MA_time, ow, -5, targ, t) #generate a maurading action ticket
            addtotimeline(MA_tic, timeline)

#Maurad action ticket handler. The bower destruction occurs if the targeted bower's owner is absent and the bower is intact. 
#Otherwise, the tau for this ticket is zero (bird immediately turns around). This ticket handler then generates a
#maurad return ticket.
def MAtickethandler(MA_tic):
    ow=MA_tic['owner'] 
    targ=MA_tic['target']
    if male_states[targ]==0 and bower_states[targ]==1: #if the bower is intact and its owner is absent
        bower_states[targ]=0
    else: #in all other cases bowerbird immediately leaves
        MA_tic['tau']=0 #COMMENT! Changing tau for this ticket as we read it bc stay is cut short by bower owner's presence
    t=MA_tic['time']
    ta=MA_tic['tau']
    MR_time=t+ta
    if MR_time<t_max: 
        MR_tau=travel_times[ow][targ]
        future_t=MR_tau+MR_time
        if future_t>t_max:
            MR_tau=MR_tau-(future_t-t_max)  
        MR_tic=ticketgenerator(MR_tau,MR_time, ow, -6, targ, t)
        addtotimeline(MR_tic, timeline)


#Maurad return ticket handler. This ticket handler then generates a ticket for stay at bower.
def MRtickethandler(MR_tic):
    ow=MR_tic['owner']
    targ=MR_tic['target'] #accesing the male the owner chose to maurad based on visit preferences
    t=MR_tic['time']
    ta=MR_tic['tau']
    SB_time=t+ta
    if SB_time<t_max:
        SB_tau=nexttau(-1)
        future_t=SB_tau+SB_time
        if future_t>t_max:
            SB_tau=SB_tau-(future_t-t_max)  
        SB_tic=ticketgenerator(SB_tau,SB_time, ow, -1, ow, t) #generate stay at bower ticket
        addtotimeline(SB_tic, timeline)

#Repair bower ticket handler. This generates a ticket for stay at bower.
def RBtickethandler(RB_tic):
    ow=RB_tic['owner']
    t=RB_tic['time']
    ta=RB_tic['tau']
    SB_time=t+ta
    if SB_time<t_max:
        SB_tau=nexttau(-1)
        future_t=SB_tau+SB_time
        if future_t>t_max:
            SB_tau=SB_tau-(future_t-t_max)  
        SB_tic=ticketgenerator(SB_tau, SB_time, ow, -1, ow, t)
        addtotimeline(SB_tic, timeline)
       
    
#Female visitation ticket handler. Females visit and mate with males if the following conditions are met: 
#(1) The male is at the bower, (2) the bower is intact, and (3) no other female is present.
#If not,the female keeps this male on her recents list, and travels to additional males with probabilities based on a preference matrix,
#If the female finds no eligeble males but her recents list is full, a longer tau is generated. 
#Her short term memory is cleared (the recents list), and she can try any of the males again. This continues 
#until she mates succusfully. Then, the male tickets for stay at bower are extended until the female leaves. 
def FVtickethandler(FV_tic, recents_list):
    targ=FV_tic['target']
    ow=FV_tic['owner']
    t=FV_tic['time']
    recents_list=d["rl{0}".format(ow)]
    r=random.random() #generate this to decide whether mating happens if other conditions are met
    if bower_states[targ]==1 and male_states[targ]!=0 and r<success_rate and t>mating_states[targ]: #if the bower is intact and the male is present
        tau_court= 0.25 #numpy.random.normal(loc=.1504, scale=.0102, size=1)[0]
        t_court_end=t+tau_court
        if male_states[targ][0]<t_court_end:
            if male_states[targ][1]==-4: #if a MT ticket, generate a new tau and new_targ (individual to be marauded by targ)
                new_targ=numpy.random.choice(list(range(nodes)), p=visit_preferences[targ]) #choosing the male to maurad based on visit preferences
                new_tau=travel_times[new_targ][targ] #tau is based on travel time
            else: #if a FG ticket, generate a new tau and set new_targ to targ
                new_tau=nexttau(male_states[targ][1]) #generate a new tau based on activity
                new_targ=targ #targ=ow (or in this case new_targ=targ) when it's an FG_tic
            future_t=new_tau+t_court_end #future t is the time when the ticket we are generating's action will end
            if future_t>t_max: #if future_t > t_max, change it so the tau hits t_max exactly
                new_tau=new_tau-(future_t-t_max)  
            tic=ticketgenerator(new_tau, t_court_end, targ, male_states[targ][1], new_targ, t) #make a ticket that has the male leaving only once female visit is over
            addtotimeline(tic, timeline)
            SB_tic=timeline[male_states[targ][2]] #this SB ticket has the wrong tau -- we must modify the timeline
            #print("this is the SB_tic with the wrong tau:")
            #print(timeline[male_states[targ][2]])
            SB_tic['tau']=t_court_end-male_states[targ][0]+SB_tic['tau'] #the new tau is generated by tacking on the extension of SB due to FV
            timeline[male_states[targ][2]]['tau']=SB_tic['tau'] #modify the ticket on the timeline itself
            #print("this is the SB_tic with the right tau??:")
            #print(timeline[male_states[targ][2]])
            male_states[targ][0]=t_court_end #now change male states so that the leave_time is accurate 
        mating_states[targ]=t_court_end
        timeline[timeholder]['tau']=tau_court
        fitness_states[targ]=fitness_states[targ]+1 #assumption: female always mates if bower is intact and male present
        success_times.append(t_court_end) #this is the time at which the mating finishes (not starts, tho it could be)       
    else: #if female does not successfully mate
        recents_list.append(targ)
        if len(recents_list)==min(max_visits, nodes): #if she's already visited her max # of males and not mated, reset recents and use nexttau
            recents_list=[]
            new_FV_targ=numpy.random.choice(list(range(nodes)))
            new_FV_time=t+nexttau(-3) #she goes to another male after waiting some time (nexttau), courting takes no time
            if new_FV_time<t_max:
                new_FV_tic=ticketgenerator(0,new_FV_time, ow, -3, new_FV_targ, t)
                addtotimeline(new_FV_tic, timeline)
        else: #otherwise find a male not in recents and use travel_times
            new_FV_targ=-1 #just so that the code goes into the while loop the first iteration
            while(new_FV_targ in recents_list or new_FV_targ==-1):
                new_FV_targ=numpy.random.choice(list(range(nodes)), p=visit_preferences[targ]) #choose a male based on preference (a function of distance)
            new_FV_time=t+travel_times[targ][new_FV_targ] #she goes directly to this male
            if new_FV_time<t_max:
                new_FV_tic=ticketgenerator(0,new_FV_time, ow, -3, new_FV_targ, t)
                addtotimeline(new_FV_tic, timeline)
    return recents_list
